## Импорт библиотек:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.api import OLS
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import VAR
from statsmodels.tools.sm_exceptions import ValueWarning, HessianInversionWarning, ConvergenceWarning
import itertools
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

# Функции для исследования:

In [3]:
def adf_test(series):
    result = adfuller(series)
    print('ADF-statistics: {:.4f}'.format(result[0]))
    print('p-value: {:.4f}'.format(result[1]))

def ARIMA_best(series, maxorder=2, check='AIC'):
    p = d = q = range(0,maxorder)
    pdq = list(itertools.product(p,d,q))
    series_list = []

    for param in pdq:
            model = ARIMA(series, order=param).fit()
            if check == 'AIC':
                series_list.append([param, model.aic])
            elif check == 'BIC':
                series_list.append([param, model.bic])

    df_model = pd.DataFrame(series_list, columns=['param', 'criteria'])
    num = df_model.index[df_model['criteria'] == df_model['criteria'].min()]
    param = df_model['param'].iloc[num].squeeze()

    model = ARIMA(series, order=param).fit()
    print(model.summary().tables[0])
    print(model.summary().tables[1])
    fig = plt.figure(figsize=(14,7.5), dpi=200)
    model.plot_diagnostics(fig=fig, auto_ylims = True)
    plt.savefig('Diagnostics for ARIMA – {}.jpeg'.format(series.name))
    plt.tight_layout()
    plt.show()
    
    return model
    
def SARIMA_best(series, season=12, check='AIC'):
    p = d = q = range(0,2)
    pdq = list(itertools.product(p,d,q))
    seasonal_pdq = [(x[0], x[1], x[2], season) for x in list(itertools.product(p,d,q))]
    series_list = []

    for param in pdq:
        for param_seasonal in seasonal_pdq:
            model = SARIMAX(series, order=param, seasonal_order=param_seasonal,
                         enforce_invertibility=False, enforce_stationarity=False).fit(disp=0)
            if check == 'AIC':
                series_list.append([param, param_seasonal, model.aic])
            elif check == 'BIC': 
                series_list.append([param, param_seasonal, model.bic])

    df_model = pd.DataFrame(series_list, columns=['param', 'param_seasonal', 'criteria'])
    num = df_model.index[df_model['criteria'] == df_model['criteria'].min()]
    param, sparam = df_model['param'].iloc[num].squeeze(), df_model['param_seasonal'].iloc[num].squeeze()

    model = SARIMAX(series, order=param, seasonal_order=sparam,
                         enforce_invertibility=False, enforce_stationarity=False).fit(disp=0)
    print(model.summary().tables[0])
    print(model.summary().tables[1])
    fig = plt.figure(figsize=(14,6), dpi=200)
    model.plot_diagnostics(fig=fig, auto_ylims = True)
    plt.savefig('Diagnostics for SARIMA – {}.jpeg'.format(series.name))
    plt.tight_layout()
    plt.show()
    
    return model
    
def check_model_values(model, series, forecast_perc=70, area_perc=40, ci=True):
    num = series.index[series == series.iloc[round(len(series)*forecast_perc/100)]]
    if len(num) > 1:
        num = num[0]
    pred = model.get_prediction(start=num[0], dynamic=False)
    pred_ci = pred.conf_int()
    df_lol = series.shift(1)
    fig, ax = plt.subplots(figsize=(12,4), dpi=200)
    df_lol.plot(ax=ax, c='b')
    pred.predicted_mean.plot(ax=ax, label='Fitted values', c='r', alpha=.7)
    
    if ci == True:
        ax.fill_between(pred_ci.index,
                       pred_ci.iloc[:, 0],
                       pred_ci.iloc[:, 1], color='k', alpha=.1, label='Confidence Interval')
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    plt.legend()
    ax.set_xlim(df_lol.index[round(len(df_lol)*area_perc/100)], 
                df_lol.index[-1])
    ax.set_title('Fitted values vs Real data – {}'.format(ax.get_legend_handles_labels()[1][0]))
    plt.savefig('Checking Model Values – {}.jpeg'.format(ax.get_legend_handles_labels()[1][0]))
    plt.show()
    mape = round(np.mean(abs(model.resid/series)*100),4)
    print('Mean Absolute Percent Error:', mape,'%')
    if mape > 5 and mape < 15:
        print('     That is ± good (> 5% and < 15%)')
    elif mape < 5:
        print('That is good! (< 5%)')
    else:
        print('     Bad! (>1 5%)')
    print('Root Mean Squared Error:', round(np.sqrt(np.mean(model.resid**2)),5))

def forecast(model, series, steps=24, ci=True, dynamic=False):
    pred = model.get_forecast(steps=steps, dynamic=dynamic)
    pred_ci = pred.conf_int()
    series = series.shift(1)
    fig, ax = plt.subplots(figsize=(12,4), dpi=200)
    series.plot(ax=ax, c='b')
    pred.predicted_mean.plot(ax=ax, label='Predicted values', c='purple', alpha=.7)
    
    if ci == True:
       ax.fill_between(pred_ci.index,
                       pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], 
                       color='#9029FD', alpha=.1, label='Confidence Interval')
    
    pred_series = pd.Series(pred.predicted_mean)
    df_forecast = pred.conf_int().join(pd.DataFrame(data=pred_series.values, index=pred.conf_int().index, 
                                                    columns=['predicted values']))
    df_forecast.to_excel('Prediction for {} – CI + predicted values.xlsx'.format(series.name))
    df_forecast.columns=['lower','upper', 'predicted values']    
    for i,j in zip(range(0,5), range(0,5)):
        ax.text(x=series.index[len(series)-1]+dt.timedelta(days=150*i), y=df_forecast['upper'][4*j]+27,
               s=round(df_forecast['upper'][4*j], 1), fontsize=8, 
                bbox=dict(boxstyle='round', facecolor='purple', alpha=0.04))
        ax.text(x=series.index[len(series)-1]+dt.timedelta(days=150*i), y=df_forecast['lower'][4*j]-27,
               s=round(df_forecast['lower'][4*j], 1), fontsize=8,
                bbox=dict(boxstyle='round', facecolor='purple', alpha=0.04))
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend(loc=2)
    ax.set_xlim(series.index[round(len(series)/2)])
    ax.set_title('Forecast – {}'.format(ax.get_legend_handles_labels()[1][0]))
    plt.savefig('Forecast Plot {}.jpeg'.format(ax.get_legend_handles_labels()[1][0]))
    plt.show()
    
def forecast_joint(model1, model2, series1, series2):
    pred1 = model1.get_forecast(steps=24)
    pred_ci1 = pred1.conf_int()
    series1 = series1.shift(1)
    fig, ax = plt.subplots(figsize=(12,4), dpi=200)
    series1.plot(ax=ax, c='orange')
    pred1.predicted_mean.plot(ax=ax, label='Predicted values 1', c='orange', ls='--', alpha=.7)
    
    ax.fill_between(pred_ci1.index,
                    pred_ci1.iloc[:, 0], pred_ci1.iloc[:, 1], 
                    color='#FDB629', alpha=.1, label='Confidence Interval 2')
    
    df_pred1 = pred1.conf_int()
    df_pred1.columns=['lower','upper']    

    pred2 = model2.get_forecast(steps=24)
    pred_ci2 = pred2.conf_int()
    series2 = series2.shift(1)
    
    series2.plot(ax=ax, c='blue')
    pred2.predicted_mean.plot(ax=ax, label='Predicted values 2', c='purple', ls='--', alpha=.7)
    
    ax.fill_between(pred_ci2.index,
                    pred_ci2.iloc[:, 0], pred_ci2.iloc[:, 1], 
                    color='#9029FD', alpha=.1, label='Confidence Interval 2')
    
    df_pred2 = pred2.conf_int()
    df_pred2.columns=['lower','upper']    
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend(loc=2)
    ax.set_xlim(series1.index[round(len(series1)/2)])
    ax.set_title('Joint Forecast of two assets')
    plt.savefig('Joint Forecast.jpeg')
    plt.show()